# Bioinformatics model for protein therapeutics

We'll use the [Therapeutics Data Commons](https://tdcommons.ai/) Python package to download open-source ([CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)) datasets that are meaningful in pharmaceutical research. In this notebook, we'll use a dataset called [TCR-Epitope Binding Affinity](https://tdcommons.ai/multi_pred_tasks/tcrepitope/).

![TCR-epitope binding](tcr-epitope-binding.png)

We show how to create a deep learning model for predicting if a T-cell receptor (TCR) and protein epitope will bind to each other. A model that can predict how well a TCR bindings to an epitope can lead to more effective treatments that use immunotherapy. For example, in anti-cancer therapies it is important for the T-cell receptor to bind to the protein marker in the cancer cell so that the T-cell (actually the T-cell's friends in the immune system) can kill the cancer cell.

We'll see how to use the open-sourced [bio-embeddings](https://docs.bioembeddings.com/v0.2.3/) Python library to get the latest state-of-the-art AI model for embedding the protein sequences. In this case, we use Facebook's open-source [Evolutionary Scale Model (ESM-1b)](https://github.com/facebookresearch/esm). These embeddings turn the protein sequences into a vector of 1,280 numbers that the computer can use in a mathematical model. The vector of numbers uniquely encodes (aka embeds) a protein sequence in the same way that the [Dewey Decimal System](https://en.wikipedia.org/wiki/Dewey_Decimal_Classification) uniquely encodes a book into a set of numbers (and letters).

Then, we'll show how to combine this embedding with a simple neural network to create a [binary classifier](https://en.wikipedia.org/wiki/Binary_classification) for the TCR-epitope binding affinity prediction (True=They Bind, False=They don't bind).

![encoder-decoder Dewey Decimal](encoder-decoder.png)


In [1]:
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas(
    desc="Embedding protein sequences"
)  # Create fancy progress bar for Pandas apply

## Get the dataset

We are using the TDC dataset for [TCR-Epitope Binding Affinity Prediction Task](https://tdcommons.ai/multi_pred_tasks/tcrepitope/).

From the TDC website:

>T-cells are an integral part of the adaptive immune system, whose survival, proliferation, activation and function are all governed by the interaction of their T-cell receptor (TCR) with immunogenic peptides (epitopes). A large repertoire of T-cell receptors with different specificity is needed to provide protection against a wide range of pathogens. This new task aims to predict the binding affinity given a pair of TCR sequence and epitope sequence.

>Weber et al.
Dataset Description: The dataset is from Weber et al. who assemble a large and diverse data from the VDJ database and ImmuneCODE project. It uses human TCR-beta chain sequences. Since this dataset is highly imbalanced, the authors exclude epitopes with less than 15 associated TCR sequences and downsample to a limit of 400 TCRs per epitope. The dataset contains amino acid sequences either for the entire TCR or only for the hypervariable CDR3 loop. Epitopes are available as amino acid sequences. Since Weber et al. proposed to represent the peptides as SMILES strings (which reformulates the problem to protein-ligand binding prediction) the SMILES strings of the epitopes are also included. 50% negative samples were generated by shuffling the pairs, i.e. associating TCR sequences with epitopes they have not been shown to bind.

>Task Description: Binary classification. Given the epitope (a peptide, either represented as amino acid sequence or as SMILES) and a T-cell receptor (amino acid sequence, either of the full protein complex or only of the hypervariable CDR3 loop), predict whether the epitope binds to the TCR.

>Dataset Statistics: 47,182 TCR-Epitope pairs between 192 epitopes and 23,139 TCRs.

>References:

1. Weber, Anna, Jannis Born, and María Rodriguez Martínez. “TITAN: T-cell receptor specificity prediction with bimodal attention networks.” Bioinformatics 37.Supplement_1 (2021): i237-i244.

2. Bagaev, Dmitry V., et al. “VDJdb in 2019: database extension, new analysis infrastructure and a T-cell receptor motif compendium.” Nucleic Acids Research 48.D1 (2020): D1057-D1062.

3. Dines, Jennifer N., et al. “The immunerace study: A prospective multicohort study of immune response action to covid-19 events with the immunecode™ open access database.” medRxiv (2020).

>Dataset License: CC BY 4.0.

>Contributed by: Anna Weber and Jannis Born.



## Download the TCR-epitope dataset

Download and split randomly into 70% training data, 10% validation data, and 20% testing data.

In [1]:
from tdc.multi_pred import TCREpitopeBinding

In [2]:
data = TCREpitopeBinding(name="weber", path="./data")  # Download the dataset
split = data.get_split(
    method="random", seed=816, frac=[0.7, 0.1, 0.2]
)  # Split the dataset

Downloading...
100%|██████████| 16.0M/16.0M [00:00<00:00, 41.0MiB/s]
Loading...
Done!


In [3]:
print(f"Train dataset size: \t\t{len(split['train']):6,d} proteins")
print(f"Validation dataset size: \t{len(split['valid']):6,d} proteins")
print(f"Test dataset size: \t\t{len(split['test']):6,d} proteins")


Train dataset size: 		33,028 proteins
Validation dataset size: 	 4,718 proteins
Test dataset size: 		 9,436 proteins


In [4]:
train_data = split["train"]
train_data


,epitope_aa,epitope_smi,tcr,tcr_full,label
0,FLKEKGGL,CC(C)C[C@H](NC(=O)CNC(=O)CNC(=O)[C@H](CCCCN)NC...,CSVWGTGKTYEQYF,SAVISQKPSRDICQRGTSLTIQCQVDSQVTMMFWYRQQPGQSLTLI...,1
1,FLKEKGGL,CC(C)C[C@H](NC(=O)CNC(=O)CNC(=O)[C@H](CCCCN)NC...,CSVWGEGRSYEQYF,SAVISQKPSRDICQRGTSLTIQCQVDSQVTMMFWYRQQPGQSLTLI...,1
2,FLKEKGGL,CC(C)C[C@H](NC(=O)CNC(=O)CNC(=O)[C@H](CCCCN)NC...,CSATILAGVPYGEQYF,GAVVSQHPSWVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLML...,1
3,FLKEKGGL,CC(C)C[C@H](NC(=O)CNC(=O)CNC(=O)[C@H](CCCCN)NC...,CASSFDREVTGELFF,GAGVSQTPSNKVTEKGKYVELRCDPISGHTALYWYRQSLGQGPEFL...,1
4,FLKEKGGL,CC(C)C[C@H](NC(=O)CNC(=O)CNC(=O)[C@H](CCCCN)NC...,CASSVGAGTEAFF,DGGITQSPKYLFRKEGQNVTLSCEQNLNHDAMYWYRQDPGQGLRLI...,1
...,...,...,...,...,...
33023,KLMNIQQKL,CC[C@H](C)[C@H](NC(=O)[C@H](CC(N)=O)NC(=O)[C@H...,CASSKPGLTDTQYF,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,0
33024,TLIGDCATV,CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H...,CASSPGQGRTHYGYTF,NAGVTQTPKFRILKIGQSMTLQCAQDMNHNYMYWYRQDPGMGLKLI...,0
33025,LLFGYPVYV,CC(C)C[C@H](N)C(=O)N[C@@H](CC(C)C)C(=O)N[C@@H]...,CASSGGSLNTEAFF,NAGVTQTPKFQVLKTGQSMTLQCAQDMNHEYMSWYRQDPGMGLRLI...,0
33026,ISPRTLNAW,CC[C@H](C)[C@H](N)C(=O)N[C@@H](CO)C(=O)N1CCC[C...,CASSPSAAMNTEAFF,TVSWYQQALGQGPQFIFQYYREEENGRGNSPPRFSGLQFPNYSSEL...,0


## What do these columns mean?

The **epitope_aa** and the **tcr_full** columns are the protein (peptide) sequences for the epitope and the T-cell receptor, respectively. The letters correspond to the [standard amino acid codes](https://en.wikipedia.org/wiki/DNA_and_RNA_codon_tables).

The **epitope_smi** column is the [SMILES](https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system) notation for the chemical structure of the epitope. We won't use this information. Instead, the ESM-1b embedder should be sufficient for the input to our binary classification model.

The **tcr** column are the first few amino acid letter codes for the T-cell receptor. It's just a label to distinguish similar TCR sequences.

The **label** column is whether the two proteins bind. 0 = No. 1 = Yes.

Our binary classification model should:
* Use the **epitope_aa** and **tcr_full** embeddings as the input
* Make a prediction if the **epitope_aa** and **tcr_full** will bind. This is the classification model's output. (0 = No, 1 = Yes)
* Use the **label** as the ground truth of the binding (i.e. what the scientific experiment says)

## The data is not shuffled

In the original datasets, the rows are sorted by label. We should randomize the row order.

In [7]:
# Randomize row order
train_data = split["train"].sample(frac = 1, random_state=816) 
validation_data = split["valid"].sample(frac = 1, random_state=816)
test_data = split["test"].sample(frac = 1, random_state=816)

## Getting embedding vectors for the protein sequences

I like using [bio-embeddings](https://github.com/sacdallago/bio_embeddings) as my library for determining embedding vectors for proteins. In this case, we use Facebook's open-source [Evolutionary Scale Model (ESM-1b)](https://github.com/facebookresearch/esm). These embeddings turn the protein sequences into a vector of 1,280 numbers that the computer can use in a mathematical model.

In [6]:
from bio_embeddings.embed import ESM1bEmbedder

In [7]:
embedder = ESM1bEmbedder()  # ESM/ESM1b (https://www.biorxiv.org/content/10.1101/622803v4)

In [8]:
# Sequence from: https://www.uniprot.org/uniprot/P58426
sequence = "DDCGKLFSGCDTNADCCEGYVCRLWCKLDW"
per_residue_embedding = embedder.embed(sequence)
per_protein_embedding = embedder.reduce_per_protein(per_residue_embedding)

print(f"The shape of each embedding vector is: {np.shape(per_protein_embedding)}")
print(f"The embedding vector for protein sequence {sequence} is: {per_protein_embedding}")


The shape of each embedding vector is: (1280,)
The embedding vector for protein sequence DDCGKLFSGCDTNADCCEGYVCRLWCKLDW is: [ 0.08437371  0.24132712 -0.05913455 ...  0.05974105  0.07406684
  0.1267224 ]


## Create embedding function

Here we create an embedding function (calling bio-embeddings) that we can use in the `pandas.apply` method.

In [9]:
def get_embedding(sequence: str, embedder) -> np.array:
    """Gets the sequence embedding for the protein sequence

    Args:
        sequence(str): The protein sequence as a string (e.g. "DDTNAWCLCDR")
        embedder:  The bio-embedder object

    Returns:
        The per protein sequence embedding vector (1280,1)
    """
    per_residue_embedding = embedder.embed(sequence)
    per_protein_embedding = embedder.reduce_per_protein(per_residue_embedding)

    return np.array(per_protein_embedding)

In [10]:
def get_both_embedding(
    sequence_A: str, sequence_B: str, embedder
) -> tuple[np.array, np.array]:
    """Gets the sequence embedding for both protein sequences

    This allows us to process the embeddings in one pass.

    Args:
        sequence_A(str): The protein sequence as string A (e.g. "DDTNAWCLCDR")
        sequence_B(str): The protein sequence as string B (e.g. "DDSEQVENCES")
        embedder:  The bio-embedder object

    Returns:
        The per protein sequence embedding vectors for A and B (1280,1), (1280,1)
    """

    return get_embedding(sequence_A, embedder), get_embedding(sequence_B, embedder)

## Get embedding for the protein sequences

Returns the embedding vectors for the **epitope** and **tcr**. We'll use these embeddings as input to our model.

In [11]:
train_data["epitope_vector"], train_data["tcr_vector"] = train_data.progress_apply(
    lambda x: get_both_embedding(x["epitope_aa"], x["tcr_full"], embedder), axis=1
)

Embedding protein sequences:   0%|          | 0/33028 [00:00<?, ?it/s]

KeyboardInterrupt: 